This notebook creates the python function for queries that include composition of available functions (i.e. tools), and might need some additional logic around combining the outputs of those functions, like mathematical operations, iterations, conditional logic etc. Model claude 2.1 is used for generation of python function


#DATA LOADING AND CLIENT INITIALISATION

In [ ]:
! pip install anthropic

In [ ]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import json

In [ ]:
ANTHROPIC_API_KEY="your api key"

In [ ]:
def read_file(path):
    with open(path, 'r') as f:
        file = json.load(f)
    return file

In [ ]:
!wget -O tools.json "https://drive.google.com/file/d/1RSC_94KKmvtWnI15RVZkLCy4eTVhxGU_/view?usp=sharing"
!wget -O examples.json "https://drive.google.com/file/d/1n9eede9tNiqfPfWyc3lx1AHgqJgxLFQ5/view?usp=drive_link"
with open('./content/examples.json', 'r') as f:
    examples = json.load(f)

with open('/content/tools.json', 'r') as f:
    tools = json.load(f)

In [ ]:
anthropic = Anthropic(api_key=ANTHROPIC_API_KEY)

PYTHON FUNCTION GENERATION


In [ ]:
def create_prompt_invalid_new(human, ai, ex):
    prompt = """
{0}: You specialize in writing python function that involve combination of tools via  mathematical operations, iterations, conditional logic etc and their respective answers that are correct, simple, and concise. In this environment you are given few example query and respective answers.
{2}
Now generate the python function such that it amswers the above query completely by taking composition of available tool, and might need some additional logic around combining the outputs of those
tools, like mathematical operations, iterations, conditional logic.
Assistant:
    """.format(human, ai, ex)
    return prompt

In [ ]:
ex_func1 = """{
  "query": "Identify similar work items as 'sprint/8' and 'issue/15' and calculate their combined count",
  "answer": [
    {
      "tool_name": "get_similar_work_items",
      "arguments": [
        {
          "argument_name": "work_id",
          "argument_value": [
            "sprint/8"
          ]
        }
      ]
    },
    {
      "tool_name": "get_similar_work_items",
      "arguments": [
        {
          "argument_name": "work_id",
          "argument_value": [
            "issue/15"
          ]
        }
      ]
    }
  ]
}"""

In [ ]:
ex_func2 = """{
  "query": "Search for Alice , find all tasks and issues, prioritize by severity and add the top 2 to the current sprint",
  "answer": [
    {
      "tool_name": "search_object_by_name",
      "arguments": [
        {
          "argument_name": "query",
          "argument_value": "Alice"
        }
      ]
    },
    {
      "tool_name": "works_list",
      "arguments": [
        {
          "argument_name": "owned_by",
          "argument_value": [
            "$$PREV[0]"
          ]
        },
        {
          "argument_name": "type",
          "argument_value": [
            "task",
            "issue"
          ]
        }
      ]
    },
    {
      "tool_name": "prioritize_objects",
      "arguments": [
        {
          "argument_name": "objects",
          "argument_value": "$$PREV[1]"
        }
      ]
    },

    {
      "tool_name": "get_sprint_id",
      "arguments": []
    },
    {
      "tool_name": "add_work_items_to_sprint",
      "arguments": [
        {
          "argument_name": "work_ids",
          "argument_value": "$$PREV[3]"
        },
        {
          "argument_name": "sprint_id",
          "argument_value": "$$PREV[4]"
        }
      ]
    }
  ]
}"""

In [ ]:
ex_func3 = """{
"query": "Get the list of all the high severity tickets of John and Alice",
"answer": [
  {
    "tool_name": "search_object_by_name",
    "arguments": [
      {
        "argument_name": "query",
        "argument_value": "John"
      }
    ]
  },
  {
    "tool_name": "search_object_by_name",
    "arguments": [
      {
        "argument_name": "query",
        "argument_value": "Alice"
      }
    ]
  },
  {
    "tool_name": "works_list",
    "arguments": [
      {
        "argument_name": "ticket.rev_org",
        "argument_value": [
          "$$PREV[0]"
        ]
      },
      {
        "argument_name": "ticket.severity",
        "argument_value": [
          "high"
        ]
      },
      {
        "argument_name": "type",
        "argument_value": [
          "ticket"
        ]
      }
    ]
  },
  {
    "tool_name": "works_list",
    "arguments": [
      {
        "argument_name": "ticket.rev_org",
        "argument_value": [
          "$$PREV[1]"
        ]
      },
      {
        "argument_name": "ticket.severity",
        "argument_value": [
          "high"
        ]
      },
      {
        "argument_name": "type",
        "argument_value": [
          "ticket"
        ]
      }
    ]
  }
]
}"""

In [ ]:
ex_func4="""{
"query": "Compare the number of p1 priority issues with the number of blocker severity tickets and return the difference in counts.",
"answer": [
  {
    "tool_name": "works_list",
    "arguments": [
      {
        "argument_name": "issue.priority",
        "argument_value": [
          "p1"
        ]
      },
      {
        "argument_name": "type",
        "argument_value": [
          "issue"
        ]
      }
    ]
  },
  {
    "tool_name": "works_list",
    "arguments": [
      {
        "argument_name": "ticket.severity",
        "argument_value": [
          "blocker"
        ]
      },
      {
        "argument_name": "type",
        "argument_value": [
          "ticket"
        ]
      }
    ]
  }
]
}"""


In [ ]:
ex_func5= """{
    "query": "Find all tasks created by user 'USER-321' and check if there are more than 10 such tasks",
    "answer": [
      {
        "tool_name": "search_object_by_name",
        "arguments": [
          {
            "argument_name": "query",
            "argument_value": "USER-321"
          }
        ]
      },
      {
        "tool_name": "works_list",
        "arguments": [
          {
            "argument_name": "created_by",
            "argument_value": [
              "$$PREV[0]"
            ]
          },
          {
            "argument_name": "type",
            "argument_value": [
              "task"
            ]
          }
        ]
      }
    ]
  }"""

In [ ]:
ex_func6 = """{
    "query": "Calculate the difference in the number of tasks between the current user and user 'USER-222'",
    "answer": [
      {
        "tool_name": "who_am_i",
        "arguments": []
      },
      {
        "tool_name": "works_list",
        "arguments": [
          {
            "argument_name": "type",
            "argument_value": [
              "task"
            ]
          },
          {
            "argument_name": "owned_by",
            "argument_value": "$$PREV[0]"
          }
        ]
      },
      {
        "tool_name": "search_object_by_name",
        "arguments": [
          {
            "argument_name": "query",
            "argument_value": "USER-222"
          }
        ]
      },
      {
        "tool_name": "works_list",
        "arguments": [
          {
            "argument_name": "type",
            "argument_value": [
              "task"
            ]
          },
          {
            "argument_name": "owned_by",
            "argument_value": "$$PREV[2]"
          }
        ]
      }
    ]
  }"""

In [ ]:
ex_func7 = """{
    "query": "Find the sprint ID for the current sprint, add all tickets with high severity to it, and return the total count of tickets added",
    "answer": [
      {
        "tool_name": "get_sprint_id",
        "arguments": []
      },
      {
        "tool_name": "works_list",
        "arguments": [
          {
            "argument_name": "ticket.severity",
            "argument_value": [
              "high"
            ]
          },
          {
            "argument_name": "type",
            "argument_value": [
              "ticket"
            ]
          }
        ]
      },
      {
        "tool_name": "add_work_items_to_sprint",
        "arguments": [
          {
            "argument_name": "sprint_id",
            "argument_value": "$$PREV[0]"
          },
          {
            "argument_name": "work_ids",
            "argument_value": "$$PREV[1]"
          }
        ]
      }
    ]
  }"""

In [ ]:
res = anthropic.completions.create(prompt = create_prompt_invalid_new(anthropic.HUMAN_PROMPT, anthropic.AI_PROMPT, ex_func7), model="claude-2.1", temperature = 0.4, max_tokens_to_sample=1500)

In [ ]:
res.completion